## Config

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta

In [2]:
from nixtlats import TimeGPT
token = '0IEAIZ41DvUuDTmpjyYgPEdOwLsnoUbaW5z4uWZm0yi6yzTU1w2xiKQeuq0nyChGv1TLxG7scd3QqNSOxqUPFVqWizI0GMCpsMMmHmOjyrJRppyYyNeIRCBniD3J1BwsRGuSg3FPrK9hbihF5bMLTp3lpLmRK9QGGzKJwxhqFo3E31P9njLgUDbB5ViGzt45MvLeBExwVhHc5ZBV5aDjhZMd1JThvBp1iNKoMceIt84mes9yH5HxJxOI0t1YlvIi'

timegpt = TimeGPT(token = token)

timegpt.validate_token()

INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

## Read data

In [29]:
df = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_yearly_processed.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30315 entries, 0 to 30314
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Series    30315 non-null  int64         
 1   N         30315 non-null  int64         
 2   NF        30315 non-null  int64         
 3   Category  30315 non-null  object        
 4   Value     18319 non-null  float64       
 5   Date      30315 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 1.4+ MB


In [41]:
print(len(df[df['Series'] == 2]))
print(len(df[df['Series'] == 1]))
print(len(df[df['Series'] == 3]))

47
47
47


## Function to perform predictions

### Perform a test to validate

In [30]:
trimmed_df = df.loc[df['Series'] == 1]
trimmed_df.dropna(subset = 'Value', inplace=True)
trimmed_df

<ipython-input-30-f7e327e7c4a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_df.dropna(subset = 'Value', inplace=True)


,Series,N,NF,Category,Value,Date
0,1,20,6,MICRO,940.66,1975-01-01
1,1,20,6,MICRO,1084.86,1976-01-01
2,1,20,6,MICRO,1244.98,1977-01-01
3,1,20,6,MICRO,1445.02,1978-01-01
4,1,20,6,MICRO,1683.17,1979-01-01
5,1,20,6,MICRO,2038.15,1980-01-01
6,1,20,6,MICRO,2342.52,1981-01-01
7,1,20,6,MICRO,2602.45,1982-01-01
8,1,20,6,MICRO,2927.87,1983-01-01
9,1,20,6,MICRO,3103.96,1984-01-01


In [31]:
# trimmed_df.drop(columns = 'Category', inplace=True)
n = int(trimmed_df['N'].iloc[0])  # Assuming N is constant within each series
nf = int(trimmed_df['NF'].iloc[0])  # Assuming NF is constant within each series

trimmed_df = trimmed_df.head(n - nf)
print(n)
print(nf)
print(len(trimmed_df))

20
6
14


In [32]:
print(type(n))
print(type(nf))

<class 'int'>
<class 'int'>


In [33]:
forecast_df = timegpt.forecast(trimmed_df, h=nf, id_col='Series', time_col='Date', target_col='Value')
forecast_df

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...


,Series,Date,TimeGPT
0,1,1989-01-01,5121.754395
1,1,1990-01-01,4974.531738
2,1,1991-01-01,4750.660156
3,1,1992-01-01,4366.261719
4,1,1993-01-01,3914.425781
5,1,1994-01-01,3427.987793


In [35]:
print(df['N'].unique())
print(df['NF'].unique())

[20 32 31 44 43 47 46 21 41 34 45 33 23 36 38 22 24 35 28 42 27 30 25 26
 40 37]
[6]


### Actual function

In [42]:
def simple_forecast_with_timegpt(df):
    forecasts_list = []  # List to hold forecast dataframes for each series
    call_count = 0  # Initialize call count
    start_time = datetime.now()  # Track the start time
    
    for series_id in df['Series'].unique():
        current_time = datetime.now()
        
        # Check if we are approaching the rate limit
        if call_count >= 200:
            # Calculate remaining time to the next minute
            time_to_next_minute = 60 - current_time.second + (1000000 - current_time.microsecond) / 1000000.0
            print(f"Pausing for {time_to_next_minute:.2f} seconds to comply with rate limit.")
            time.sleep(time_to_next_minute)  # Pause execution
            
            # Reset the call count and start time for the new minute
            call_count = 0
            start_time = datetime.now()
        
        series_df = df[df['Series'] == series_id].sort_values('Date')  # Filter and sort by date
        series_df.dropna(subset=['Value'], inplace=True)  # Drop missing values
        
        n = int(series_df['N'].iloc[0])  # Assuming N is constant within each series
        nf = int(series_df['NF'].iloc[0])  # Assuming NF is constant within each series
        
        # Ensure the dataframe passed does not use more than (N - NF) examples
        trimmed_df = series_df.head(n - nf)
        
        try:
            forecast_df = timegpt.forecast(trimmed_df, h=nf, id_col='Series', time_col='Date', target_col='Value')
            forecasts_list.append(forecast_df)  # Collect results
        # Call TimeGPT model with nf steps forecast
        except Exception as e:
            print(f"Error encountered for series {series_id}: {e}")
        
        call_count += 1  # Increment call count after each API call
        
        # Check if a minute has passed; if so, reset the counter
        if (datetime.now() - start_time).seconds >= 60:
            call_count = 0
            start_time = datetime.now()

    # Concatenate all forecast DataFrames
    final_forecasts_df = pd.concat(forecasts_list, ignore_index=True)
    
    return final_forecasts_df

## Run the function

In [43]:
### Took 32 Minutes to run! ###
### Cost 30.00 USD ###
### 645 calls made ###
# all_forecasts = simple_forecast_with_timegpt(df)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: YS-JAN
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.tim

In [44]:
all_forecasts

,Series,Date,TimeGPT
0,1,1989-01-01,5121.754395
1,1,1990-01-01,4974.531738
2,1,1991-01-01,4750.660156
3,1,1992-01-01,4366.261719
4,1,1993-01-01,3914.425781
...,...,...,...
3865,645,1988-01-01,5939.305664
3866,645,1989-01-01,7121.730469
3867,645,1990-01-01,6801.107422
3868,645,1991-01-01,6733.273926


In [46]:
all_forecasts['Date'] = pd.to_datetime(all_forecasts['Date'])
all_forecasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3870 entries, 0 to 3869
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Series   3870 non-null   int64         
 1   Date     3870 non-null   datetime64[ns]
 2   TimeGPT  3870 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 90.8 KB


In [47]:
# all_forecasts.to_parquet('M3_yearly_simple_forecasts.parquet', compression='snappy', engine='pyarrow')